In [1]:
from datasets import load_dataset

dataset = load_dataset('TheNoob3131/mosquito-data')

Using custom data configuration TheNoob3131--mosquito-data-bf1f4d4d5f427763
Reusing dataset csv (C:\Users\Aswin.Surya24\.cache\huggingface\datasets\TheNoob3131___csv\TheNoob3131--mosquito-data-bf1f4d4d5f427763\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset['train'].features['Photo']
from datasets import Image
dataset = dataset.cast_column('Photo', Image(decode=True))

In [3]:
dataset = dataset.class_encode_column('Classification')
dataset = dataset.rename_column('Classification', 'labels')

Parameter 'function'=<function Dataset.class_encode_column.<locals>.cast_to_class_labels at 0x00000210D3C24160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Casting to class labels:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

['aedes', 'culex', 'neither']


In [5]:
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
print(id2label)

{0: 'aedes', 1: 'culex', 2: 'neither'}


In [6]:
from transformers import ConvNextFeatureExtractor
model_name_or_path = 'facebook/convnext-base-224-22k'
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)
print(feature_extractor)

ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [7]:
from torchvision.transforms import (
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

transform = Compose(
    [
     RandomResizedCrop(feature_extractor.size),
     RandomHorizontalFlip(),
     ToTensor(),
     normalize
    ]
)

def train_transforms(examples):
  examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["Photo"]]

  return examples

#def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
  #  inputs = feature_extractor([x for x in example_batch['Photo']], return_tensors='pt')

    # Don't forget to include the labels!
  #  inputs['labels'] = example_batch['labels']
  #  return inputs

In [8]:
prepared_ds = dataset.with_transform(train_transforms)

In [9]:
prepared_ds["train"][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [10]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.tensor([example["labels"] for example in examples])

  return {"pixel_values": pixel_values, "labels": labels}

dataloader = DataLoader(prepared_ds["train"], collate_fn=collate_fn, batch_size=4, shuffle=True)

In [11]:
batch = next(iter(dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [12]:
len(labels)

3

In [13]:
from transformers import ConvNextForImageClassification
#print(len(labels))
model = ConvNextForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model.train()

Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-base-224-22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ConvNextForImageClassification(
  (convnext): ConvNextModel(
    (embeddings): ConvNextEmbeddings(
      (patch_embeddings): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (layernorm): ConvNextLayerNorm()
    )
    (encoder): ConvNextEncoder(
      (stages): ModuleList(
        (0): ConvNextStage(
          (downsampling_layer): Identity()
          (layers): Sequential(
            (0): ConvNextLayer(
              (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (layernorm): ConvNextLayerNorm()
              (pwconv1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELUActivation()
              (pwconv2): Linear(in_features=512, out_features=128, bias=True)
              (drop_path): Identity()
            )
            (1): ConvNextLayer(
              (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (layernorm): ConvNextLayerNorm()
     

In [14]:
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

ConvNextForImageClassification(
  (convnext): ConvNextModel(
    (embeddings): ConvNextEmbeddings(
      (patch_embeddings): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (layernorm): ConvNextLayerNorm()
    )
    (encoder): ConvNextEncoder(
      (stages): ModuleList(
        (0): ConvNextStage(
          (downsampling_layer): Identity()
          (layers): Sequential(
            (0): ConvNextLayer(
              (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (layernorm): ConvNextLayerNorm()
              (pwconv1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELUActivation()
              (pwconv2): Linear(in_features=512, out_features=128, bias=True)
              (drop_path): Identity()
            )
            (1): ConvNextLayer(
              (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
              (layernorm): ConvNextLayerNorm()
     

In [15]:
prepared_ds["train"][0]

{'MHM Id': 17233.0,
 'labels': 0,
 'Container': 'plant clumps (bamboo etc)',
 'Userid': 2538037,
 'latitude': -22.8854,
 'longitude': 47.7708,
 'date': '9/19/2019',
 'Photo': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4160x3120 at 0x210D8B79220>,
 'pixel_values': tensor([[[-1.9638, -1.9980, -1.9980,  ..., -2.0323, -2.0323, -1.9809],
          [-2.0152, -1.9980, -1.9980,  ..., -2.0323, -1.9980, -1.9809],
          [-1.9638, -2.0152, -1.9980,  ..., -2.0323, -2.0152, -1.9980],
          ...,
          [-1.9467, -1.9467, -1.9638,  ..., -2.0323, -2.0323, -2.0837],
          [-1.9295, -1.9124, -1.9809,  ..., -2.0323, -2.0494, -2.0837],
          [-1.9295, -1.9467, -1.9467,  ..., -2.0323, -2.0323, -2.0323]],
 
         [[-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          [-2.0182, -2.0182, -2.0357,  ..., -2.0182, -2.0357, -2.0357],
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0357, -2.0357],
          ...,
          [-2.0182, -2.0182, -2.0182,  ..., 

In [16]:
from datasets import load_metric
import numpy as np


def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
  

    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./convnext-base-mosquito",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [20]:
prepared_ds['train'].features

{'MHM Id': Value(dtype='float64', id=None),
 'labels': ClassLabel(num_classes=3, names=['aedes', 'culex', 'neither'], id=None),
 'Container': Value(dtype='string', id=None),
 'Userid': Value(dtype='int64', id=None),
 'latitude': Value(dtype='float64', id=None),
 'longitude': Value(dtype='float64', id=None),
 'date': Value(dtype='string', id=None),
 'Photo': Image(decode=True, id=None)}

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

In [22]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

C:\Users\Aswin.Surya24\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7107
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3556


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,0.962400,0.842519,0.359512,0.599593,0.449504,0.599593
200,0.907300,0.838380,0.562151,0.604827,0.564939,0.604827
300,0.757800,0.797866,0.596527,0.628671,0.608520,0.628671
400,0.733900,0.805577,0.612129,0.637976,0.599691,0.637976
500,0.655700,0.957681,0.584131,0.571969,0.568025,0.571969
600,0.684300,0.831955,0.641886,0.631870,0.625072,0.631870
700,0.561800,0.820480,0.644464,0.661820,0.637497,0.661820
800,0.694400,0.767711,0.649529,0.666764,0.636907,0.666764
900,0.684000,0.751415,0.633549,0.651643,0.632334,0.651643
1000,0.746100,0.933204,0.626335,0.566444,0.590080,0.566444


***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
C:\Users\Aswin.Surya24\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./convnext-base-mosquito\checkpoint-100
Configuration saved in ./convnext-base-mosquito\checkpoint-100\config.json
Model weights saved in ./convnext-base-mosquito\checkpoint-100\pytorch_model.bin
Feature extractor saved in ./convnext-base-mosquito\checkpoint-100\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
C:\Users\Aswin.Surya24\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to con

***** train metrics *****
  epoch                    =               4.0
  total_flos               =      2093958364GF
  train_loss               =            0.5627
  train_runtime            = 1 day, 9:08:48.33
  train_samples_per_second =             0.238
  train_steps_per_second   =              0.03


In [23]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6563
  eval_f1                 =     0.6355
  eval_loss               =     0.7498
  eval_precision          =     0.6386
  eval_recall             =     0.6563
  eval_runtime            = 0:37:35.67
  eval_samples_per_second =      1.525
  eval_steps_per_second   =      0.191
